In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
import configparser 
import datetime

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata

In [2]:
#Берем данные по БД

config = configparser.ConfigParser()
config.read('config.ini')
conn_string = config.get('DATABASE', 'connection_url')

In [ ]:
#Чтение из БД

In [3]:
table = 'house_prices_train'
query = f''' 
select * from {table}
'''
def reading_from_db(query,table):
    db = create_engine(conn_string)
    
    with db.connect() as conn:
        df = pd.read_sql(query, con=conn)
        print('Reading is complete')
    return df


In [4]:
df = reading_from_db(query,table)

Reading is complete


In [ ]:
#Генерация

In [5]:
metadata = Metadata.detect_from_dataframe(
    data=df,
    table_name='house_prices')

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df)

synthetic_data = synthesizer.sample(num_rows=10)

/opt/anaconda3/lib/python3.11/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [6]:
synthetic_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,982268879,185,RL,69.0,11584,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,515,4,2007,New,Partial,275890,2024-10-09 11:28:36
1,804069452,108,RL,101.0,8706,Pave,NaN,IR3,Lvl,AllPub,...,NaN,NaN,NaN,575,4,2009,WD,Partial,246788,2024-10-09 11:28:36
2,839063981,48,RL,NaN,11230,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,2064,8,2009,WD,Normal,225499,2024-10-09 11:28:36
3,82149587,124,RL,71.0,6560,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,8,9,2006,WD,Partial,164671,2024-10-09 11:28:36
4,204237279,93,RL,91.0,14465,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,7,8,2009,WD,Normal,83705,2024-10-09 11:28:36
5,975382569,184,RL,NaN,1595,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,3303,4,2007,WD,Normal,91521,2024-10-09 11:28:36
6,952095692,33,RL,NaN,11269,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2006,WD,Normal,116355,2024-10-09 11:28:36
7,152067842,143,RL,44.0,8909,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,52,8,2009,WD,Normal,287552,2024-10-09 11:28:36
8,536692407,64,RL,54.0,8696,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,6185,7,2008,WD,Abnorml,244868,2024-10-09 11:28:36
9,86284612,98,RL,95.0,11287,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,3695,6,2008,WD,Normal,158684,2024-10-09 11:28:36


In [7]:
#Сохранение модели для генерации случайных данных 

synthesizer.save(
    filepath='my_synthesizer.pkl'
)


In [11]:
synthetic_data['dt'] = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
#Баг с обучением
synthetic_data['3SsnPorch'] = 0

In [ ]:
#Загрузка модели для генерации случайных данных

synthesizer = GaussianCopulaSynthesizer.load(
    filepath='my_synthesizer.pkl'
)

In [12]:
synthetic_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,dt
0,982268879,185,RL,69.0,11584,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,515,4,2007,New,Partial,275890,2024-10-11 11:52:48
1,804069452,108,RL,101.0,8706,Pave,NaN,IR3,Lvl,AllPub,...,NaN,NaN,NaN,575,4,2009,WD,Partial,246788,2024-10-11 11:52:48
2,839063981,48,RL,NaN,11230,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,2064,8,2009,WD,Normal,225499,2024-10-11 11:52:48
3,82149587,124,RL,71.0,6560,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,8,9,2006,WD,Partial,164671,2024-10-11 11:52:48
4,204237279,93,RL,91.0,14465,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,7,8,2009,WD,Normal,83705,2024-10-11 11:52:48
5,975382569,184,RL,NaN,1595,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,3303,4,2007,WD,Normal,91521,2024-10-11 11:52:48
6,952095692,33,RL,NaN,11269,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2006,WD,Normal,116355,2024-10-11 11:52:48
7,152067842,143,RL,44.0,8909,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,52,8,2009,WD,Normal,287552,2024-10-11 11:52:48
8,536692407,64,RL,54.0,8696,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,6185,7,2008,WD,Abnorml,244868,2024-10-11 11:52:48
9,86284612,98,RL,95.0,11287,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,3695,6,2008,WD,Normal,158684,2024-10-11 11:52:48


In [ ]:
#Запись в БД

In [ ]:
#Берем данные по БД

config = configparser.ConfigParser()
config.read('config.ini')
conn_string = config.get('DATABASE', 'connection_url')

In [10]:
#Функция чтения и записи в БД

def upload_to_db(df_in,db_table_out):
    try:

        db = create_engine(conn_string) 
        with db.connect() as conn:
            
            df_in.to_sql(db_table_out, con=conn, if_exists='append',index=False) 
            print('Uploaded')
    finally:
        print('OK')


In [13]:
df_in = synthetic_data
db_table_out = 'house_prices_generator'

upload_to_db(df_in,db_table_out)

Uploaded
OK


In [ ]:
df.dtypes

In [ ]:
synthetic_data.dtypes

In [ ]:
synthesizer.sample(num_rows=10)

In [ ]:
df[['3SsnPorch']].value_counts()